# GPT API Fine Tuning for Named Entity Recognition (NER)

Try chatgpt for your task before fine-tuning. If chatgpt is not enough, then fine-tune it. Do not forget that chat-gpt may be sufficient for most simple tasks.

To fine-tune the GPT-3.5 model, you need to convert each observation in your data to the following message template (Jsonline Format):

In [ ]:
# """{'messages': [{'role': 'system', 'content': 'Given a text, provide the following fields in a JSON format, where applicable:\
#                                              'person (only name/surname)', 'time (week, month, summer etc.)', 'currency (USD, EURO etc.)',\
#                                               and 'place (country, city etc.)'.'},
#                  {'role': 'user', 'content': 'The White House is located in Washington, D.C.'},
#                  {'role': 'assistant', 'content': "{'Person': 'null', 'Time': 'null', 'Currency': 'null', 'Place': 'Washington, D.C.'}"}]}"""

# system: the instructions you give to the model.
# user: the feature/independent variable/text of each observation in your data (X)
# assistant: the dependent variable/target/label of each observation in your data (y).

In [ ]:
!pip install --upgrade openai

In [ ]:
!pip install tiktoken

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY']=userdata.get('openai_key')

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']
)

We're getting 20 observations ready for NER fine tuning.

In [ ]:
prompt= ["After spending several months studying abroad in Spain, John Wick returned home with a newfound appreciation for the Spanish language and culture.",
         "Despite the fact that I had saved up 100.000 USD, I still found it difficult to afford a house in the city.",
         "When I was a child, my grandparents would take me on long walks through the countryside, and those memories are still some of my most cherished.",
         "After years of working in the corporate world, I decided to start my own business, and I've never looked back.",
         "The ancient ruins we visited in Rome were so awe-inspiring that I found myself speechless.",
         "Despite the fact that I had been practicing for months, my piano recital was a disaster, and I was humiliated.",
         "When she traveled to Thailand, Mary was amazed by the beauty and serenity of the Buddhist temples.",
         "My grandmother's estate, which had been passed down through several generations of our family, was the site of many cherished family gatherings.",
         "When I was a student, I spent a semester studying abroad in France, and it was one of the most enriching experiences of my life.",
         "After my grandfather passed away, my family and I spent weeks going through his belongings and reminiscing about his life.",
         "The White House is located in Washington, D.C.",
         "I need to exchange dollars for euros before I travel to Europe.",
         "The exchange rate between the pound and the euro is favorable.",
         "As a child, I used to spend my summers at my grandparents' farm, and those days are some of my most cherished memories.",
         "After years of working for a multinational corporation, I decided to quit my job and travel the World.",
         "My great-grandfather fought in World War II, and his stories about the war have been passed down through the generations of our family.",
         "When I was in college, I spent a semester studying abroad in China, and it was an eye-opening experience.",
         "My best friend and I have been planning a trip to South America for years, and we're finally going to make it happen this summer.",
         "My parents own a small motel in a charming seaside town, and they've been running it for over 20 years.",
         "Johnson and I decided to take a road trip across the United States during the summer after we graduated from college."]

In [ ]:
# You can edit the output however you like. We wanted the output to be in dictionary format, so we formatted it this way.

target= [{"Person": "John Wick", "Time": "several months", "Currency": "null", "Place": "Spain"},
         {"Person": "null", "Time": "null", "Currency": "USD", "Place": "null"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "years", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "Rome"},
         {"Person": "null", "Time": "months", "Currency": "null", "Place": "null"},
         {"Person": "Mary", "Time": "null", "Currency": "null", "Place": "Thailand"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "semester", "Currency": "null", "Place": "France"},
         {"Person": "null", "Time": "weeks", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "Washington, D.C."},
         {"Person": "null", "Time": "null", "Currency": ["dollars", "euros"], "Place": "Europe"},
         {"Person": "null", "Time": "null", "Currency": ["pound", "euro"], "Place": "null"},
         {"Person": "null", "Time": ["summers", "those days"], "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "years", "Currency": "null", "Place": "World"},
         {"Person": "null", "Time": "null", "Currency": "null", "Place": "null"},
         {"Person": "null", "Time": "semester", "Currency": "null", "Place": "China"},
         {"Person": "null", "Time": ["years", "summer"], "Currency": "null", "Place": "South America"},
         {"Person": "null", "Time": "over 20 years", "Currency": "null", "Place": "null"},
         {"Person": "Johnson", "Time": "summer", "Currency": "null", "Place": "United States"}]

In [ ]:
my_dict={"text":prompt, "target":target}
df=pd.DataFrame(my_dict)
df

,text,target
0,After spending several months studying abroad ...,"{'Person': 'John Wick', 'Time': 'several month..."
1,Despite the fact that I had saved up 100.000 U...,"{'Person': 'null', 'Time': 'null', 'Currency':..."
2,"When I was a child, my grandparents would take...","{'Person': 'null', 'Time': 'null', 'Currency':..."
3,"After years of working in the corporate world,...","{'Person': 'null', 'Time': 'years', 'Currency'..."
4,The ancient ruins we visited in Rome were so a...,"{'Person': 'null', 'Time': 'null', 'Currency':..."
5,Despite the fact that I had been practicing fo...,"{'Person': 'null', 'Time': 'months', 'Currency..."
6,"When she traveled to Thailand, Mary was amazed...","{'Person': 'Mary', 'Time': 'null', 'Currency':..."
7,"My grandmother's estate, which had been passed...","{'Person': 'null', 'Time': 'null', 'Currency':..."
8,"When I was a student, I spent a semester study...","{'Person': 'null', 'Time': 'semester', 'Curren..."
9,"After my grandfather passed away, my family an...","{'Person': 'null', 'Time': 'weeks', 'Currency'..."


In [ ]:
X=df["text"]
y=df["target"]

In [ ]:
system="""Extract the labels delimited by XML tags from text delimited by XML tags and return them in dict type.
<labels>'Person': <person names and surnames>, 'Time': <word or words indicating time \
like this week, last month, summer, next month etc.>', 'Currency':<all currencies like USD, EURO, etc.>, \
'Place':<settlement like country, city, district etc.> </labels>"""
system
# commonly used delimiters:
# triple quotes: """
# triple backticks: '''
# riple dashes: ___
# angle brackets : <>
# curly brackets: {}
# XML tags: <tag> </tag> ---> <text>This is sentence</text>

"Extract the labels delimited by XML tags from text delimited by XML tags and return them in dict type.\n<labels>'Person': <person names and surnames>, 'Time': <word or words indicating time like this week, last month, summer, next month etc.>', 'Currency':<all currencies like USD, EURO, etc.>, 'Place':<settlement like country, city, district etc.> </labels>"

In [ ]:
#system="""Extract {labels} from text and return them in dict type.
#labels: 'Person (name, surname)', 'Time (this week, last month, summer, next month etc.)', 'Currency (USD, EURO, etc.)', 'Place (country, city, etc.)'.""" #

In [ ]:
import json
import pandas as pd

def convert_df_to_dict(X, y):
  """Converts X and y to dictionary format.

  Args:
    X: text
    y: label

  Returns:
    A dataframe(as X and y) in dictionary format.
  """
  #df = pd.read_csv(csv_file)
  json_lines = []
  for i, j in zip(X, y):
    json_lines.append({
        "messages": [
            {
                "role": "system",
                "content": system
            },
            {
                "role": "user",
                "content": i
            },
            {
                "role": "assistant",
                "content": str(j) # We convert assistant content in dictionary format to string.
            }                     # All content must be strings, otherwise you will get an error in the Format validation section.
        ]
    })

  return json_lines

In [ ]:
train=convert_df_to_dict(X, y)

In [ ]:
train[0]

{'messages': [{'role': 'system',
   'content': "Extract the labels delimited by XML tags from text delimited by XML tags and return them in dict type.\n<labels>'Person': <person names and surnames>, 'Time': <word or words indicating time like this week, last month, summer, next month etc.>', 'Currency':<all currencies like USD, EURO, etc.>, 'Place':<settlement like country, city, district etc.> </labels>"},
  {'role': 'user',
   'content': 'After spending several months studying abroad in Spain, John Wick returned home with a newfound appreciation for the Spanish language and culture.'},
  {'role': 'assistant',
   'content': "{'Person': 'John Wick', 'Time': 'several months', 'Currency': 'null', 'Place': 'Spain'}"}]}

In [ ]:
import json

with open("train.jsonl", "w") as f:
    for item in train:
        f.write(json.dumps(item) + "\n")

## Data preparation and analysis for chat model fine-tuning

**Data loading**, **Format validation**, **Token Counting Utilities**, **Data Warnings and Token Counts**, ve **Cost Estimation** bölümleri için aşağıdaki kod blokları OpenAI tarafından hazırlanmıştır. Ve ince ayardan önce kullanılması tavsiye edilir.

Bu kod blokları, bir chatgpt modeline ince ayar yapmak için kullanılan datayı önceden işlemek ve analiz etmek için bir araç görevi görür. Biçim hatalarını kontrol eder, temel istatistikler sağlar ve ince ayar maliyetini belirlemek için gerekli olan token sayılarını tahmin eder.

In [ ]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

## Data loading

We first load the chat dataset from an example JSONL file.

In [ ]:
data_path = "/content/train.jsonl" # data_path variable specifies the path to the jsonl file.

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f: # with open() opens the file and loads each line as a dict object using json.loads().

    dataset = [json.loads(line) for line in f] # we assign all dict objects to the dataset list.

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 20
First example:
{'role': 'system', 'content': "Extract the labels delimited by XML tags from text delimited by XML tags and return them in dict type.\n<labels>'Person': <person names and surnames>, 'Time': <word or words indicating time like this week, last month, summer, next month etc.>', 'Currency':<all currencies like USD, EURO, etc.>, 'Place':<settlement like country, city, district etc.> </labels>"}
{'role': 'user', 'content': 'After spending several months studying abroad in Spain, John Wick returned home with a newfound appreciation for the Spanish language and culture.'}
{'role': 'assistant', 'content': "{'Person': 'John Wick', 'Time': 'several months', 'Currency': 'null', 'Place': 'Spain'}"}


## Format validation

We can perform various error checks to verify that each observation in the dataset conforms to the format expected by the fine-tuning API. Errors are categorized according to their nature for easier debugging.

1.**Data Type Check: Checks whether each entry in the dataset is a dictionary (dict). Error type: data_type.

2.**Presence of Message List: Checks if there is a message list for each entry. Error type: missing_messages_list.

3.**Message Keys Check: Checks for the presence of “role” and “content” keys in messages. Error type: message_missing_key.

4.**Unrecognized Keys in Messages: Checks if a message has keys other than “role”, “content” and name(system, user, assistant). Error type: message_unrecognized_key.

5.**Role Validation: Confirms whether the “roles” are “system”, “user”, or “assistant”. Error type: unrecognized_role.

6.**Content Validation (content validation)**: confirms that the content is a string expression. Error type: missing_content.

7.**Assistant Message Presence: **: checks if the message contains content belonging to the assistant. Error type: example_missing_assistant_message.

The following code performs these checks and prints the number of any errors found. This is useful for debugging and checking if the dataset is ready for the next steps.

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# This function checks if the data is ready for fine tuning. It returns an error for missing or invalid data.
# returns the message “No error found” if there is no problem with the data.

No errors found


## Token Counting Utilities

Lets define a few helpful utilities to be used in the rest of the notebook.

In [ ]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.05)}, {np.quantile(values, 0.95)}")

## Data Warnings and Token Counts

With the following analyses, we can identify potential problems in the data, missing messages and provide statistical information about message and token counts.

1.**Missing System/User Messages**: Counts observations that are missing “system” or “user” messages. These messages are critical for identifying the assistant's behavior and initiating the conversation.

2.**Number of Messages Per Example**: Summarizes the distribution of the number of messages (system, user, assistant) in each observation and provides information about the dialog complexity. It checks if each message has 3 separate parts: system, user, assistant. If so, the min, max, mean, median, 5% and 95% of the number of parts in each message will always be 3. Otherwise, these values will be different from 3.

3.**Total Tokens Per Example**: Calculates and summarizes the distribution of the total number of tokens in each observation. It is important to understand the fine-tuning costs. This provides information about the total number of tokens and their distribution (mean, median, minimum, maximum, etc.) in each message, including system, user and assistant parts.

4.**Tokens in Assistant's Messages**: calculates the number of tokens in assistant's messages in each observation and summarizes this distribution (mean, median, minimum, maximum, etc.). Provides information about the assistant.

5.**Token Limit Warnings**:  Checks if any observation exceeds the maximum token limit (4096 tokens), because such observations will be clipped during fine-tuning, which will result in data loss.

In [ ]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 142, 159
mean / median: 151.25, 151.5
p5 / p95: 143.9, 159.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 24, 29
mean / median: 25.6, 24.5
p5 / p95: 24.0, 29.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


## Cost Estimation

In this section, we estimate the total number of tokens that will be used for fine-tuning, which allows us to approximate the cost. It is worth noting that as the number of tokens increases, the duration of the fine-tuning process will also increase.

In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

current_price=8 # Price per one million tokens, please check the openai webpage for the current price.
print(f"Estimate total costs ~{(n_epochs * n_billing_tokens_in_dataset)/1000000 * current_price}")

# At least 10 observations must be given to the model for fine tuning. Otherwise the model will return an error. However, it is recommended to give the model a minimum of 100 and a maximum of 25000 observations for fine tuning.

# GPT 3.5-Turbo model works with 3 epochs by default for fine tuning. However, this script will suggest you how many epochs you should train, from a minimum of 1 to a maximum of 25, based on the number of observations in your data.

# Returns the number of tokens to process over the entire training and the cost of fine-tuning based on that number of tokens.

# NOTE: This section will only apply to training data.

Dataset has ~3025 tokens that will be charged for during training
By default, you'll train for 5 epochs on this dataset
By default, you'll be charged for ~15125 tokens
Estimate total costs ~0.121


In [ ]:
training_response = client.files.create(
                                        file=open("/content/train.jsonl", "rb"),
                                        purpose="fine-tune") # We prepare Train data for fine tuning.

training_file_id = training_response.id # We pull the ID created for the ready train data.

print("Training file id:", training_file_id)

Training file id: file-QETofvgMzM3vaF3XZ7HywbLP


In [ ]:
suffix_name = "NER"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id, # The ID of the train data is given to the fine-tuning model. The ID of the train data must be provided
    model="gpt-3.5-turbo", # For now the only model available is the GPT-3.5-turbo. We expect GPT-4 to be available in the near future.
    suffix=suffix_name, # We can add a suffix of our choice to the name of the fine-tuned model, but we cannot specify the exact name of the model.
    hyperparameters={"n_epochs":5}, # We are only allowed to set the Epoch, batch size parameter. We are not allowed to edit other parameters.
                                    # Since the recommended number of epochs is 3, we set epoch to 3.
)

job_id = response.id # The ID of fine-tune model.

print(response)

FineTuningJob(id='ftjob-aT6Qa2uYF6puBDD0i21tHPIB', created_at=1713810490, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=[], seed=2062274915, status='validating_files', trained_tokens=None, training_file='file-QETofvgMzM3vaF3XZ7HywbLP', validation_file=None, integrations=[], user_provided_suffix='NER')


In [ ]:
job_id

'ftjob-aT6Qa2uYF6puBDD0i21tHPIB'

In [ ]:
client.fine_tuning.jobs.retrieve(job_id) # We get general information about fine tuning.

FineTuningJob(id='ftjob-aT6Qa2uYF6puBDD0i21tHPIB', created_at=1713810490, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:ner:9GsmiFLb', finished_at=1713810866, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=['file-fTM0sgqWT3VOWfnworyX3dTm'], seed=2062274915, status='succeeded', trained_tokens=14925, training_file='file-QETofvgMzM3vaF3XZ7HywbLP', validation_file=None, integrations=[], user_provided_suffix='NER')

In [ ]:
client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10) # Displays the events of the fine-tuning process in the last 10 steps.

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-H6XABXfSnbzVMSy6fnmlyJu6', created_at=1713810871, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-6yc2zMygNvLLiijQd3PkeOcc', created_at=1713810868, level='info', message='New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal:ner:9GsmiFLb', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-mDGIe4Bctt7j5CXKBS9YqgqI', created_at=1713810868, level='info', message='Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:ner:9GsmhsGl:ckpt-step-80', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-BRFRAp0sI7SU1Na8cmavpdfQ', created_at=1713810868, level='info', message='Checkpoint created at step 60 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:ner:9GsmhhNB:ckpt-step-60', object='fine_tuning.job.event', data={}, t

In [ ]:
response= client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=30)

events=response.data

events.reverse() # We reverse the order of the fine-tuning steps using the reverse() function, with the first steps of the fine-tuning process at the beginning.

for event in events:
  print(event.message)

# We get the training loss scores for each step.

Step 75/100: training loss=0.00
Step 76/100: training loss=0.00
Step 77/100: training loss=0.00
Step 78/100: training loss=0.00
Step 79/100: training loss=0.00
Step 80/100: training loss=0.48
Step 81/100: training loss=0.00
Step 82/100: training loss=0.00
Step 83/100: training loss=0.46
Step 84/100: training loss=0.00
Step 85/100: training loss=0.00
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.00
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.38
Step 94/100: training loss=0.00
Step 95/100: training loss=0.00
Step 96/100: training loss=0.00
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 60 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:ner:9GsmhhNB:ckpt-step-60
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-tur

In [ ]:
#client.fine_tuning.jobs.cancel(job_id)

# You can cancel the fine-tuning process you have started with this code. However, the fine-tuning process will start in 5-10 minutes,
# so you need to cancel it before that. If you don't, you won't be able to cancel it.

FineTuningJob(id='ftjob-nKraJjw1ggsXvBT8TNkrSoZw', created_at=1713810376, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=[], seed=483893693, status='cancelled', trained_tokens=None, training_file='file-QETofvgMzM3vaF3XZ7HywbLP', validation_file=None, integrations=[], user_provided_suffix='NER')

## Prediction

In [ ]:
client.fine_tuning.jobs.retrieve(job_id) # First we get general information about fine-tuning. from here we will draw the fine-tune model name

FineTuningJob(id='ftjob-aT6Qa2uYF6puBDD0i21tHPIB', created_at=1713810490, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:ner:9GsmiFLb', finished_at=1713810866, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-b88r78OSLP9wnPnDMsswFKhJ', result_files=['file-fTM0sgqWT3VOWfnworyX3dTm'], seed=2062274915, status='succeeded', trained_tokens=14925, training_file='file-QETofvgMzM3vaF3XZ7HywbLP', validation_file=None, integrations=[], user_provided_suffix='NER')

In [ ]:
model_name= client.fine_tuning.jobs.retrieve(job_id).fine_tuned_model # we pull the model name.
model_name

'ft:gpt-3.5-turbo-0125:personal:ner:9GsmiFLb'

In [ ]:
completion = client.chat.completions.create(
  model=model_name,
  messages=[
    {"role": "system", "content": system},
    {"role": "user", "content": "My grandmother was a talented seamstress, and she taught me how to sew when I was young, which sparked a lifelong passion for fashion and design."}
  ]
)

print(completion.choices[0].message.content)

{'Person': 'null', 'Time': 'null', 'Currency': 'null', 'Place': 'null'}


In [ ]:
completion = client.chat.completions.create(
  model=model_name,
  messages=[
    {"role": "system", "content": system},
    {"role": "user", "content": "While I was on summer vacation, Joseph earned 100 dollars and 100 euros in Turkey and returned to America."}
  ]
)

print(completion.choices[0].message.content)

{'Person': 'Joseph', 'Time': 'summer', 'Currency': ['dollars', 'euros'], 'Place': 'Turkey'}


In [ ]:
completion = client.chat.completions.create(
  model=model_name,
  messages=[
    {"role": "system", "content": system},
    {"role": "user", "content": "Johnson Walker and Maria got married last month. They bought a house in London for 100,000 sterling pounds."}
  ]
)

print(completion.choices[0].message.content)

{'Person': 'Johnson Walker and Maria', 'Time': 'last month', 'Currency': 'sterling pounds', 'Place': 'London'}


END OF THE PROJECT